# Assignment Part 1 Brief 

## Deadline: Friday, December 3, 2021 at 15:00 hrs

## Number of marks available for Part 1: 25

## Scope: Sessions 1 to 5

### Please READ the whole assignment first, before starting to work on it.

### How and what to submit

A. A **Jupyter Notebook** with the code in all the cells executed and outputs displayed.

B. Name your Notebook **COM4509-6509_Assignment_Part1_Username_XXXXXX.ipynb** where XXXXXX is your username such as such as abc18de.  

C. Upload a .zip file to Blackboard before the deadline that contains the Jupyter Notebook in B and any other files requested for the solution of Part 2 of the Assignment (Dr Lu will be in charge of releasing this part). 

D. **NO DATA UPLOAD**: Please do not upload the data files used in this Notebook. We have a copy already. 


### Assessment Criteria 

* Being able to use numpy and pandas to preprocess a dataset.

* Being able to use numpy to build a machine learning pipeline for supervised learning. 

* Being able to follow the steps involved in an end-to-end project in machine learning.

* Being able to use scikit-learn to design a machine learning model pipeline


### Late submissions

We follow Department's guidelines about late submissions, i.e., a deduction of 5% of the mark each working day the work is late after the deadline. NO late submission will be marked one week after the deadline because we will release a solution by then. Please read [this link](https://sites.google.com/sheffield.ac.uk/comughandbook/general-information/assessment/late-submission) if you are taking COM4509 or read [this link](https://sites.google.com/sheffield.ac.uk/compgtstudenthandbook/menu/assessment/late-submission) if you are taking COM6509. 

### Use of unfair means 

**"Any form of unfair means is treated as a serious academic offence and action may be taken under the Discipline Regulations."** (from the students Handbook). Please carefully read [this link](https://sites.google.com/sheffield.ac.uk/comughandbook/general-information/assessment/unfair-means) on what constitutes Unfair Means if not sure, for COM4509. If you are taking COM6509, please read [this link](https://sites.google.com/sheffield.ac.uk/compgtstudenthandbook/menu/referencing-unfair-means) if you are not sure what is Unfair means. If you still have questions, please ask your Personal tutor or the Lecturers. 

# A dataset of air quality

The dataset you will use in this assignment comes from a popular machine learning repository that hosts open source datasets for educational and research purposes, the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php). We are going to use regularised ridge regression and random forests for predicting air quality. The description of the dataset can be found [here](https://archive.ics.uci.edu/ml/datasets/Air+Quality).

In [1]:
import urllib.request
doq = "https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip"
pat_sav = "./AirQualityUCI.zip"
urllib.request.urlretrieve(doq, pat_sav)
#urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip")

('./AirQualityUCI.zip', <http.client.HTTPMessage at 0x7fe1015b9d00>)

In [2]:
import zipfile
zip = zipfile.ZipFile('./AirQualityUCI.zip', 'r')
for name in zip.namelist():
    zip.extract(name, '.')

In [3]:
# The .csv version of the file has some typing issues, so we use the excel version
import pandas as pd 
air_quality_full = pd.read_excel('./AirQualityUCI.xlsx', usecols=range(2,15))

We can see some of the rows in the dataset 

In [4]:
air_quality_full.sample(5)

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
512,1.1,1034.00,71,4.086652,715.50,50.0,1085.0,55.0,1405.00,891.25,10.675,67.224998,0.862988
3490,-200.0,866.00,-200,2.264475,603.25,48.0,1077.0,55.0,1458.75,588.50,25.750,50.875000,1.660524
2065,3.6,1115.75,-200,17.541988,1226.00,-200.0,791.5,-200.0,1938.25,1037.50,25.675,31.750000,1.031738
6167,4.0,1509.25,-200,25.401786,1439.25,1080.0,459.0,199.0,1577.00,2077.50,8.500,52.625000,0.585614
23,2.9,1383.00,150,11.151581,1019.75,243.0,1008.0,135.0,1718.75,1104.00,9.775,67.624998,0.818501


The target variable corresponds to the CO(GT) variable of the first column. The following columns correspond to the variables in the feature vectors, *e.g.*, PT08.S1(CO) is $x_1$ up until AH which is $x_D$. The original dataset also has a date and a time columns that we are not going to use in this assignment.

### Removing instances 

The dataset has missing values tagged with a -200 value. To simplify the design of the machine learning models in this assignment, we perform the following two operations to the dataset right from the beginning:

* we will remove the rows for which the target variable has missing values. We are doing supervised learning so we need all our data observations to have known target values.

* we will remove features with more than 20% of missing values. 

The code below performs both operations

In [5]:
# We first remove the rows for which there are missing values in the target feature
air_quality = air_quality_full.loc[air_quality_full.iloc[:, 0]!=-200, :]

In [6]:
# We now remove the columns (features) for which there are more that 20% of missing values
import numpy as np
ndata, ncols = np.shape(air_quality) # number of data observations and number of columns in the dataframe
pmissing = np.empty(ncols)         # An empty vector that will keep the percentage of missing values per feature
for i in range(ncols):
    pmissing[i] = (air_quality.iloc[:, i]==-200).sum()/ndata # Computes the percentage of missing values per column
air_quality = air_quality.loc[:, pmissing < 0.2]

### Splitting the dataset 

Before designing any machine learning model, we need to set aside the test data. We will use the remaining training data for fitting the model. *It is important to remember that the test data has to be set aside before preprocessing*. 

Any preprocessing that you do has to be done only on the training data and several key statistics need to be saved for the test stage.  Separating the dataset into training and test before any preprocessing has happened help us to recreate the real world scenario where we will deploy our system and for which the data will come without any preprocessing.

Furthermore, we are going to use *hold-out validation* for validating our predictive model so we need to further separate the training data into a training set and a validation set.

We split the dataset into a training set, a validation set and a test set. The training set will have 70% of the total observations, the validation set 15% and the test set, the remaining 15%. For making the random selections of the training and validation sets **make sure that you use a random seed that corresponds to the last five digits of your student UCard**. In the code below, I have used 55555 as an example of my random seed.

In [7]:
np.random.seed(69752)                 # Make sure you use the last five digits of your student UCard as your seed
index = np.random.permutation(ndata)  # We permute the indexes 
N = np.int64(np.round(0.70*ndata))    # We compute N, the number of training instances
Nval = np.int64(np.round(0.15*ndata)) # We compute Nval, the number of validation instances   
Ntest = ndata - N - Nval              # We compute Ntest, the number of test instances
data_training_unproc = air_quality.iloc[index[0:N], :].copy() # Select the training data
data_val_unproc = air_quality.iloc[index[N:N+Nval], :].copy() # Select the validation data
data_test_unproc = air_quality.iloc[index[N+Nval:ndata], :].copy() # Select the test data

The assigment is divided into two sections. In the **first section**, you will design a regularised ridge regression model trained with stochastic gradient descent. You will write all the code from scratch. You should not use any library that already implements any of the routines considered in this section, for example, scikit-learn. In the **second section**, you will design a random forests model and you are allowed to use scikit-learn in this section.

When writing your code, you will find out that there are operations that are repeated at least twice. We will assign marks for use of Python functions and for commenting your code. The marks will be assigned as:

* Did you include Python functions to solve the question and avoid repeating code? (**1 mark**)
* Did you comment your code to make it readable to others? (**1 mark**)

# 1. Using regularised ridge regression to predict air quality (10 marks)

**DO NOT USE scikit-learn or any other machine learning library for the questions on this section. You are meant to write Python code from scratch. You can use Pandas and Numpy. Using scikit-learn or any other machine learning library for the questions in this section will give ZERO marks. No excuse will be accepted.**

Regularisation is a technique commonly used in Machine Learning to prevent overfitting. It consists on adding terms to the objective function such that the optimisation procedure avoids solutions that just learn the training data. Popular techniques for regularisation in Supervised Learning include [Lasso Regression](https://en.wikipedia.org/wiki/Lasso_(statistics)), [Ridge Regression](https://en.wikipedia.org/wiki/Tikhonov_regularization) and the [Elastic Net](https://en.wikipedia.org/wiki/Elastic_net_regularization). 

In this part of the Assignment, you will be looking at Ridge Regression and implementing equations to optimise the objective function using the update rules for stochastic gradient descent. You will use those update rules for making predictions on the Air Quality dataset.

## 1.1 Ridge Regression

Let us start with a data set for training $\mathcal{D} = \{\mathbf{y}, \mathbf{X}\}$, where the vector $\mathbf{y}=[y_1, \cdots, y_N]^{\top}$ and $\mathbf{X}$ is the design matrix from Lab 4, this is, 

\begin{align*}
    \mathbf{X} = 
                \begin{bmatrix}
                        1 & x_{1,1} & \cdots & x_{1, D}\\
                        1 & x_{2,1} & \cdots & x_{2, D}\\
                   \vdots &  \vdots\\
                        1 & x_{N,1} & \cdots & x_{N, D}
                \end{bmatrix}
               = 
               \begin{bmatrix}
                      \mathbf{x}_1^{\top}\\
                       \mathbf{x}_2^{\top}\\
                          \vdots\\
                        \mathbf{x}_N^{\top}
                \end{bmatrix}.
\end{align*}

Our predictive model is going to be a linear model

$$ f(\mathbf{x}_i) = \mathbf{w}^{\top}\mathbf{x}_i,$$

where $\mathbf{w} = [w_0\; w_1\; \cdots \; w_D]^{\top}$.

The **objective function** we are going to use has the following form

$$ E(\mathbf{w}, \lambda) = \frac{1}{N}\sum_{n=1}^N (y_n - f(\mathbf{x}_n))^2 + \frac{\lambda}{2}\sum_{j=0}^D w_j^2,$$

where $\lambda>0$ is known as the *regularisation* parameter.

This objective function was studied in Lecture 4. 

The first term on the rhs is what we call the "fitting" term whereas the second term in the expression is the regularisation term. Given $\lambda$, the two terms in the expression have different purposes. The first term is looking for a value of $\mathbf{w}$ that leads the squared-errors to zero. While doing this, $\mathbf{w}$ can take any value and lead to a solution that it is only good for the training data but perhaps not for the test data. The second term is regularising the behavior of the first term by driving the $\mathbf{w}$ towards zero. By doing this, it restricts the possible set of values that $\mathbf{w}$ might take according to the first term. The value that we use for $\lambda$ will allow a compromise between a value of $\mathbf{w}$ that exactly fits the data (first term) or a value of $\mathbf{w}$ that does not grow too much (second term).

This type of regularisation has different names: ridge regression, Tikhonov regularisation or $\ell_2$ norm regularisation. 

## 1.2 Optimising the objective function with respect to $\mathbf{w}$

There are two ways we can optimise the objective function with respect to $\mathbf{w}$. The first one leads to a closed form expression for $\mathbf{w}$ and the second one using an iterative optimisation procedure that updates the value of $\mathbf{w}$ at each iteration by using the gradient of the objective function with respect to $\mathbf{w}$,
$$
\mathbf{w}_{\text{new}} = \mathbf{w}_{\text{old}} - \eta \frac{d E(\mathbf{w}, \lambda)}{d\mathbf{w}},
$$
where $\eta$ is the *learning rate* parameter and $\frac{d E(\mathbf{w}, \lambda)}{d\mathbf{w}}$ is the gradient of the objective function.

It can be shown (this is a question in the Exercise Sheet 4) that a closed-form expression for the optimal $\mathbf{w}_*$ is given as

\begin{align*}            
            \mathbf{w}_*& = \left(\mathbf{X}^{\top}\mathbf{X} + \frac{\lambda N}   
                                     {2}\mathbf{I}\right)^{-1}\mathbf{X}^{\top}\mathbf{y}.
\end{align*}

Alternatively, we can find an update equation for $\mathbf{w}_{\text{new}}$ using gradient descent leading to:

\begin{align*}
   \mathbf{w}_{\text{new}} & = \mathbf{w}_{\text{old}} - \eta \frac{d E(\mathbf{w}, \lambda)}
                              {d\mathbf{w}},\\
                           & = \mathbf{w}_{\text{old}} +  \frac{2\eta}{N}\sum_{n=1}^N   
                               \left(y_n - \mathbf{x}_n^{\top}\mathbf{w}_{\text{old}}\right)\mathbf{x}_n  
                       - \eta\lambda\mathbf{w}_{\text{old}}\\
                           & = (1 - \eta\lambda)\mathbf{w}_{\text{old}} + \frac{2\eta}
                               {N}\sum_{n=1}^N   
                               \left(y_n - \mathbf{x}_n^{\top}\mathbf{w}_{\text{old}}\right)\mathbf{x}_n
\end{align*}

## 1.3 Preprocessing the data

As mentioned before, the dataset has missing values tagged with a -200 value. Before doing any work with the training data, we want to make sure that we deal properly with the missing values. Furthermore, once we have dealt with the missing values, we want to standardise the training data. 

### Question 1.a: Missing values and standardisation (2 marks)

* For all the other features with missing values, use the mean value of the non-missing values for that feature to perform imputation. Save these mean values, you will need them when performing the validation stage (**1 mark**).

* Once you have imputed the missing data, we need to standardise the input vectors. Standardise the training data by substracting the mean value for each feature and dividing the result by the standard deviation of each feature. Keep the mean values and standard deviations, you will need them at validation time (**1 mark**).

#### Question 1.a Answer

In [8]:
import numpy as np
data_training = data_training_unproc.copy()
data_val = data_val_unproc.copy()

#define a function to calculate the mean of a feature and use the mean to replace the missing value
def cal_mean(data_training):
    nrows, ncols = np.shape(data_training) 
    feature_average = np.empty(ncols)
    for i in range(ncols):#calcuelate the mean of the feature
        s = 0
        index = 0
        average = 0
        for j in range(nrows):
            if data_training.iloc[j, i] == -200:
                j += 1
            else:
                s += data_training.iloc[j,i]
                index += 1
        feature_average[i] = s / index
    for u in range(ncols): #if the data is missing then let it be the average number of this feature
        for v in range(nrows):
            if data_training.iloc[v, u] == -200:
                data_training.iloc[v, u] == feature_average[u]
    return feature_average

#define a function to calculate the standard deviation
def std_dev(data_training):
    nrows, ncols = np.shape(data_training)
    standard_deviation = np.empty(ncols)
    for i in range(ncols):
        standard_deviation[i] = np.std(data_training.iloc[:, i])
    return standard_deviation

#define a function to standardise the input vectors
def std_input(data):
    nrows, ncols = np.shape(data)
    for i in range(ncols): #standardise the input vectors
        for j in range(nrows):
            data.iloc[j, i] = (data.iloc[j, i] - mean[i]) / std[i]
    return data

#standardising the input vectors
mean = cal_mean(data_training)
std = std_dev(data_training)
data_training = std_input(data_training)

In [9]:
print(mean, std, data_training)

[2.13842144e+00 1.10822389e+03 1.02014960e+01 9.45123833e+02
 2.51407008e+02 8.27002042e+02 1.14210754e+02 1.44490662e+03
 1.04092287e+03 1.78544616e+01 4.90430037e+01 9.94207016e-01] [  1.42864078 338.85578115  43.14845782 345.81503905 228.46176633
 324.92515175  85.3216156  476.39049756 465.73646616  44.9584154
  53.25157342  40.69096132]         CO(GT)  PT08.S1(CO)  C6H6(GT)  PT08.S2(NMHC)   NOx(GT)  PT08.S3(NOx)  \
7437  0.393086     0.319043  0.100546       0.546177  0.474009     -0.803268   
6756 -0.096890    -3.860710 -4.871588      -3.311377  0.952426     -3.160734   
2878  0.183096     0.276743  0.154718       0.755104 -0.168111     -0.481656   
477  -0.166887     0.041393 -0.052108      -0.138438 -0.583936      0.504725   
9054  0.393086     0.135828 -0.092381      -0.356762 -0.283667     -0.527820   
...        ...          ...       ...            ...       ...           ...   
761  -0.236884    -0.063372 -0.066995      -0.216514 -0.632084      0.573972   
6422  1.163049   

## 1.4 Training and validation stages

We have now curated our training data by removing data observations and features with a large amount of missing values. We have also normalised the feature vectors. We are now in a good position to work on developing the prediction model and validating it. We will use gradient descent for iterative optimisation. 

We first organise the dataframe into the vector of targets $\mathbf{y}$, call it `yTrain`, and the design matrix $\mathbf{X}$, call it `XTrain`.

In [10]:
#set a vector of targets y called yTrain
yTrain = (np.array(data_training_unproc['CO(GT)'].values)).reshape(-1,1)
#set a matrix of x called xTrain and add a column of one
x = np.array(data_training.iloc[:, 1:].values)
xTrain = np.hstack((np.ones((x.shape[0], 1)), np.array(x)))

### Question 1.b: finding the optimal $\mathbf{w}$ with stochastic gradient descent (3 marks)

Use gradient descent to iteratively compute the value of $\mathbf{w}_{\text{new}}$. Instead of using all the training set in `XTrain` and `yTrain` to compute the gradient, use a subset of $S$ instances in `XTrain` and `yTrain`. This is sometimes called *minibatch gradient descent* where $S$ is the size of the minibacth. When using gradient descent with minibatches, you need to find the best values for three parameters: $\eta$, the learning rate, $S$, the number of datapoints in the minibatch and $\gamma$, the regularisation parameter.

* In this question we will use the validation data. So before proceeding to the next steps, make sure that you:  replace the missing values on each feature variables with the mean value you computed with the training data; standardise the validation data using the means and standard deviations computed from the training data (**1 mark**).
    
* Create a grid of values for the parameters $\gamma$ and $\eta$ using `np.logspace` and a grid of values for $S$ using `np.linspace`. Because you need to find three parameters, start with five values for each parameter in the grid and see if you can increase it. Make sure you understand what is the meaning of `np.logspace` and `np.linspace`. Notice that you can use negative values for `start` in `np.logspace` (**1 mark**).

* For each value that you have of $\gamma$, $\eta$ and $S$ from the previous step, use the training set to compute $\mathbf{w}$ using minibatch gradient descent and then measure the RMSE over the validation data. For the validation data, make sure you preprocess it before applying the prediction model over it. For the minibatch gradient descent choose to stop the iterative procedure after $200$ iterations (**1 mark**).

* Choose the values of $\gamma$, $\eta$ and $S$ that lead to the lowest RMSE and save them. You will use them at the test stage.

#### Question 1.b Answer

In [11]:
import random

#set a grid of values for the parameters 𝛾， 𝜂 and S
def get_eta (min, max, num = 5):
    eta = np.logspace(min, max, 5)
    return eta
def get_gamma(min, max, num = 5):
    gamma = np.logspace(min, max, 5)
    return gamma
def get_S(min, max, num = 5):
    s = np.linspace(min, max, 5)
    return s

#define a function named cal_w to calculate the w using gradient descent
def cal_w(eta, gamma, S, w, y, x):
    Sum = np.array(np.zeros(x.shape[1]).reshape(-1,1));
    for i in range(200):
        for j in range(S):
            a = y[j][:] - np.dot(x[j][:], w)
            Sum = Sum + a * x[j][:].reshape(-1, 1)
            w = (1 - eta * gamma) * w + (2 * eta / S) * Sum
    return w 

#define a function named cal_RMSE to calculate RMSE
def cal_RMSE(w, x, y, m):
    sum = 0
    for p in range(m):
        sum += (y[p] - np.dot(x[p], w)) ** 2
    RMSE = np.sqrt(sum / m)
    return RMSE

In [12]:
#preprocess the validation data
import numpy as np

data_training = data_training_unproc.copy()
data_val = data_val_unproc.copy()
nrows, ncols = np.shape(data_val) 
feature_average = np.empty(ncols)
data_val = std_input(data_val)
yVal = (np.array(data_val_unproc['CO(GT)'].values)).reshape(-1,1)
x = np.array(data_val.iloc[:, 1:].values)
xVal = np.hstack((np.ones((x.shape[0], 1)), np.array(x)))

In [13]:
# get every number of eta, gamma and S to get the accurate w through calculate RMSE
eta = get_eta(-1, 0.0005, 5)
gamma = get_gamma(0.1, 0.06, 5)
s = get_S(100, 300, 5)

#define a function to get the eta, gamma and s that lead to the lowest RMSE
w_old = np.array(np.ones(xTrain.shape[1]).reshape(-1,1));
w_new = np.array(np.ones(xTrain.shape[1]).reshape(-1,1));

def get_par(e, g, s, wold, wnew, yT, xT, xV, yV):
    boundary = 50
    geteta, getgamma, gets = 0, 0, 0
    w = np.array(np.ones(xTrain.shape[1]).reshape(-1,1));
    for i in range(len(e)):
        for j in range(len(g)):
            for k in range(len(s)):
                S = int(s[k])
                wnew = cal_w(e[i], g[j], S, wold, yT, xT)
                rmse = cal_RMSE(wnew, xV, yV, S)
                if rmse < boundary:
                    boundary = rmse
                    geteta = e[i]
                    getgamma = g[j]
                    gets = s[k]
                    w = wnew
    return boundary, geteta, getgamma, gets, w

RMSE, ETA, GAMMA, S, W = get_par(eta, gamma, s, w_old, w_new, yTrain, xTrain, xVal, yVal)
print("the lowest RMSE is:", RMSE)
print("the eta that lead to the lowest RMSE is:", ETA)
print("the gamma that lead to the lowest RMSE is:", GAMMA)
print("the s that lead to the lowest RMSE is:", S)

the lowest RMSE is: [0.47101967]
the eta that lead to the lowest RMSE is: 0.1
the gamma that lead to the lowest RMSE is: 1.2589254117941673
the s that lead to the lowest RMSE is: 100.0


## 1.5 Test stage 

We now know which one is the best model, according to the validation data. We will now put together the training data and the validation data, perform the preprocessing as we did before, this is, treat the missing values and standardise the inputs. We will train the model again using the minibatch stochastic gradient descent and finally compute the RMSE over the test data.


### Question 1.c: combine the original training and original validation data and perform the preprocessing again to this new data (2 marks)

Put together the original training and validation dataset and perform the same preprocessing steps than before, these are: 

* for each feature, impute the missing values with the mean values of the non-missing values (**1 mark**) 

* stardardise the new training set (**1 mark**).

#### Question 1.c Answer

In [14]:
#combine the original training and original validation data
data_comb_unproc = pd.concat([data_training_unproc, data_val_unproc], axis = 0)
data_comb = data_comb_unproc.copy()
#preprocess the combination data
mean1 = cal_mean(data_comb)
print(mean1)
data_comb = std_input(data_comb)
print(data_comb)

[2.14432010e+00 1.10881347e+03 1.02318703e+01 9.45808611e+02
 2.53117455e+02 8.28100331e+02 1.14366499e+02 1.44429185e+03
 1.04191517e+03 1.78120197e+01 4.90094367e+01 9.91059430e-01]
        CO(GT)  PT08.S1(CO)  C6H6(GT)  PT08.S2(NMHC)   NOx(GT)  PT08.S3(NOx)  \
7437  0.393086     0.319043  0.100546       0.546177  0.474009     -0.803268   
6756 -0.096890    -3.860710 -4.871588      -3.311377  0.952426     -3.160734   
2878  0.183096     0.276743  0.154718       0.755104 -0.168111     -0.481656   
477  -0.166887     0.041393 -0.052108      -0.138438 -0.583936      0.504725   
9054  0.393086     0.135828 -0.092381      -0.356762 -0.283667     -0.527820   
...        ...          ...       ...            ...       ...           ...   
4278  2.562981     0.856636  0.466164       1.772989 -1.975854     -1.088719   
310  -0.096890     0.070166 -0.035089      -0.052409 -0.540165      0.360846   
2326  0.393086    -0.126821  0.148167       0.730524 -0.360704     -0.189281   
3221  0.253093  

### Question 1.d: Preprocess the test data, train the model and predict over the test data (3 marks)

Preprocess the test data and train a new model using the new training set. Finally, report the RMSE over the test set:

* Preprocess the test data by imputing the missing data and standardising it (**1 mark**). 

* Use the best values of $\gamma$, $\eta$ and $S$ found in the validation set and train a new regularised linear model with stochastic gradient descent (**1 mark**).

* Report the RMSE over the test data (**1 mark**).

#### Question 1.d Answer

In [15]:
#preprocess the test data
data_comb_unproc = pd.concat([data_training_unproc, data_val_unproc], axis = 0)
data_comb = data_comb_unproc.copy()
data_comb = std_input(data_comb)
yComb = (np.array(data_comb_unproc['CO(GT)'].values)).reshape(-1,1)
x = np.array(data_comb.iloc[:, 1:].values)
xComb = np.hstack((np.ones((x.shape[0], 1)), np.array(x)))

#preprocess the test data
data_test = data_test_unproc.copy()
data_test = std_input(data_test)
yTest = (np.array(data_test_unproc['CO(GT)'].values)).reshape(-1,1)
x = np.array(data_test.iloc[:, 1:].values)
xTest = np.hstack((np.ones((x.shape[0], 1)), np.array(x)))

#start to train a new model and report the RMSE
# get every number of eta, gamma and S to get the accurate w through calculate RMSE
eta = get_eta(-1, 0.0005, 5)
gamma = get_gamma(0.1, 0.06, 5)
s = get_S(100, 300, 5)

#define a function to get the eta, gamma and s that lead to the lowest RMSE
w_old = np.array(np.ones(xComb.shape[1]).reshape(-1,1));
w_new = np.array(np.ones(xComb.shape[1]).reshape(-1,1));

RMSE, ETA, GAMMA, S, W = get_par(eta, gamma, s, w_old, w_new, yComb, xComb, xTest, yTest)
print("the lowest RMSE is:", RMSE)
print("the eta that lead to the lowest RMSE is:", ETA)
print("the gamma that lead to the lowest RMSE is:", GAMMA)
print("the s that lead to the lowest RMSE is:", S)

the lowest RMSE is: [0.54407247]
the eta that lead to the lowest RMSE is: 0.3164098537558792
the gamma that lead to the lowest RMSE is: 1.1481536214968828
the s that lead to the lowest RMSE is: 250.0


# 2. Random forests (13 marks)

**USE scikit-learn for the questions on this section.**

In section 1, you used a regularised ridge regression model trained with SGD to create a linear predictive model. In this part of the assignment, you will use **scikit-learn** to train a random forest for regression over the air quality dataset.

## 2.1 Preprocessing the data

As mentioned before, the dataset has missing values tagged with a -200 value. Before doing any work with the training data, we want to make sure that we deal properly with the missing values. Furthermore, once we have dealt with the missing values, we want to standardise the training data. 

### Question 2.a: Pipeline for missing values and standardisation (3 marks)

* Employ the `SimpleImputer` method in Scikit-learn to impute the missing values in each column using the mean value of the non-missing values, instead (**1 mark**).

* Standardise the data by substracting the mean value for each feature and dividing the result by the standard deviation of each feature. Employ the `StandardScaler` method (**1 mark**).

* Create a `Pipeline` that you can use to preprocess the data by filling missing values and then standardising the features (**1 mark**).

#### Question 2.a Answer

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
air_quality_vice = air_quality.copy()
imputer = SimpleImputer(missing_values = -200, strategy = 'mean')
scaler = StandardScaler()
pipeline = Pipeline(steps = [('i', imputer), ('s', scaler)])

### Question 2.b: Use the Pipeline to fit the training data and transform the validation data (2 marks)

In the previous question, you created a `Pipeline` for applying a `SimpleImputer` and a `StandardScaler`. Use the Pipeline to fit the training data (**1 mark**) and transform the validation data (**1 mark**).

#### Question 2.b Answer

In [17]:
# Use the Pipeline to fit the training data and transform the validation data 
data_training = data_training_unproc.copy()
data_val = data_val_unproc.copy()

data_training_set_attributes = data_training.drop('CO(GT)', axis=1)
data_training_set_labels = data_training['CO(GT)']
data_val_set_attributes = data_val.drop('CO(GT)', axis=1)
data_val_set_labels = data_val['CO(GT)']

data_training_set_attributes = pipeline.fit_transform(data_training_set_attributes )
data_val_set_attributes = pipeline.transform(data_val_set_attributes)
print(data_training_set_attributes[:, 1:3].mean())
print(data_val_set_attributes)

-2.24855296126614e-16
[[ 0.46608834  0.14672217  0.304455   ... -1.33463278  0.7805096
  -1.00869984]
 [-0.97674693 -0.5965125  -0.53046992 ... -0.34692454  1.77204045
   0.85313612]
 [-1.40959751 -1.21869693 -1.5621916  ...  0.92993891  0.79654899
   2.73316217]
 ...
 [-0.20329261  0.89491232  0.98447552 ...  2.00353471 -1.76684986
   0.15956935]
 [ 0.7913176   0.61057557  0.73896668 ...  1.31929626 -0.60471737
   1.42771742]
 [-0.2943568  -0.30368491 -0.17389755 ...  2.23829437 -1.86600296
   0.20632866]]


## 2.2 Random forest to predict air quality 

We now use random forests to predict air quality. Remember that the tree ensemble in random forests is built by training individual regression trees on different subsets of the training data and using a subset of the available features. For regression, the prediction is the average of the individual predictions of each tree. Some of the parameters required in the Random Forest implementation in Scikit-learn include:

Some of the additional parameters required in the Random Forest implementation in Scikit-learn include

> **n_estimators** the total number of trees to train<p>
**max_features** number of features to use as candidates for splitting at each tree node. <p>
    **boostrap**: Whether bootstrap samples are used when building trees. If False, the whole dataset is used to build each tree.<p>
   **max_samples**: If bootstrap is True, the number of samples to draw from X to train each base estimator.

### Question 2.c: train a random forest (4 marks)

In this question, you will train a random forest for predicting over the validation data. Use cross-validation over the validation data to select the best set of paramaters for the random forest regressor. Parameters to include in your exploration are **n_estimators**, **max_features** and **max_samples**. Use `np.linspace` or `np.logspace` to define ranges of values to explore for each parameter and create a grid to be assessed over the validation data. Make sure you use the same validation data that was given to you.

* Use `PredefinedSplit` to tell the cross validator which instances to use for training and which ones for validation (**1 mark**).

* Create a grid of values to explore that include a range of at least five values for each parameter **n_estimators**, **max_features** and **max_samples** (**1 mark**). 

* Train a random forest for regression model that uses the grid of parameters you created before. Use `GridSearchCV` to find the best set of parameters by performing cross-validation over the predefined split. (**1 mark**).

* Print the best values in your grid for **n_estimators**, **max_features** and **max_samples** (**1 mark**).

#### Question 2.c Answer

In [18]:
from sklearn.model_selection import PredefinedSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit
test_fold = np.zeros((np.shape(data_comb)[0], 1))
test_fold[0:np.shape(data_training)[0]] = -1
ps = PredefinedSplit(test_fold)

whole_train_set_attributes = np.vstack((data_training_set_attributes, data_val_set_attributes))
whole_train_set_labels = np.hstack((data_training_set_labels, data_val_set_labels))

n_estimators = np.linspace(105, 115, 5).astype(int).tolist()
max_features = np.linspace(4, 6, 5).astype(int).tolist()
max_samples = np.linspace(500, 4000, 5).astype(int).tolist()
param_grid = dict(n_estimators = n_estimators, max_features = max_features, max_samples = max_samples)
grid_regression = GridSearchCV(RandomForestRegressor(), param_grid = param_grid, cv=ps, scoring='neg_mean_squared_error')
grid_regression.fit(whole_train_set_attributes, whole_train_set_labels)

regr = RandomForestRegressor(n_estimators = grid_regression.best_params_["n_estimators"], max_features = grid_regression.best_params_["max_features"], max_samples = grid_regression.best_params_["max_samples"])
regr

RandomForestRegressor(max_features=4, max_samples=4000, n_estimators=110)

### Question 2.d: train a new model over the whole training data and report the prediction over the test set (4 marks)


Now that we have identified the best paramaters of the regression model, we use these parameters to train a new model over the whole training data (`data_training` plus `data_val`). We apply this model to the test set and report the performance.

* Create a new preprocessing pipeline for taking care of the missing values and standardisation over the whole training data (**1 mark**).

* Apply the created preprocessing pipeline to the test data (**1 mark**).

* Fit a random forest regression model to the training data using the best parameters found in Question 2.c (**1 mark**).

* Compute the RMSE over the test data and report the result (**1 mark**).

#### Question 2.d Answer

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

# preprocess the data
imputer = SimpleImputer(missing_values = -200, strategy = 'mean')
scaler = StandardScaler()
pipeline = Pipeline(steps = [('i', imputer), ('s', scaler)])

data_training = data_training_unproc.copy()
data_val = data_val_unproc.copy()
data_comb_unproc = pd.concat([data_training_unproc, data_val_unproc], axis = 0)
data_comb = data_comb_unproc.copy()
data_test = data_test_unproc.copy()

data_newtraining_set_attributes = data_comb.drop('CO(GT)', axis=1)
data_newtraining_set_labels = data_comb['CO(GT)']
data_test_set_attributes = data_test.drop('CO(GT)', axis=1)
data_test_set_labels = data_test['CO(GT)']

data_newtraining_set_attributes = pipeline.fit_transform(data_newtraining_set_attributes )
data_test_set_attributes = pipeline.transform(data_test_set_attributes)

whole_train_set_attributes = np.vstack((data_newtraining_set_attributes, data_test_set_attributes))
whole_train_set_labels = np.hstack((data_newtraining_set_labels, data_test_set_labels))

# regr = GradientBoostingRegressor(n_estimators=grid_regression.best_params_["n_estimators"], max_features=grid_regression.best_params_["max_features"], max_samples=grid_regression.best_params_["max_samples"])
regr.fit(data_newtraining_set_attributes, data_newtraining_set_labels)
data_test_set_predictions = regr.predict(data_test_set_attributes)

# Computing the RMSE for the validation dataset
error_mod = np.sqrt(mean_squared_error(data_test_set_labels, data_test_set_predictions))
print('The RMSE on the validation data is :',error_mod)

The RMSE on the validation data is : 0.44401668665089594
